In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## ഘട്ടം 1: ഘടനാപരമായ ഔട്ട്പുട്ടുകൾക്കുള്ള Pydantic മോഡലുകൾ നിർവചിക്കുക

ഈ മോഡലുകൾ ഏജന്റുകൾ തിരികെ നൽകുന്ന **schema** നിർവചിക്കുന്നു. Pydantic ഉപയോഗിച്ച് `response_format` ഉപയോഗിക്കുന്നത് ഉറപ്പാക്കുന്നു:
- ✅ ടൈപ്പ്-സേഫ് ഡാറ്റ എക്സ്ട്രാക്ഷൻ
- ✅ സ്വയമേവ പരിശോധന
- ✅ സ്വതന്ത്ര ടെക്സ്റ്റ് പ്രതികരണങ്ങളിൽ നിന്ന് പാഴ്സിംഗ് പിഴവുകൾ ഇല്ല
- ✅ ഫീൽഡുകൾ അടിസ്ഥാനമാക്കി എളുപ്പത്തിലുള്ള കണ്ടീഷണൽ റൂട്ടിംഗ്


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## ഘട്ടം 2: ഹോട്ടൽ ബുക്കിംഗ് ടൂൾ സൃഷ്ടിക്കുക

**availability_agent** റൂമുകൾ ലഭ്യമാണോ എന്ന് പരിശോധിക്കാൻ വിളിക്കുന്ന ടൂൾ ഇതാണ്. `@ai_function` ഡെക്കറേറ്റർ ഉപയോഗിക്കുന്നത്:
- ഒരു Python ഫംഗ്ഷനെ AI-നു വിളിക്കാവുന്ന ടൂളാക്കി മാറ്റാൻ
- LLM-നു വേണ്ടി JSON schema സ്വയമേവ സൃഷ്ടിക്കാൻ
- പാരാമീറ്റർ പരിശോധന കൈകാര്യം ചെയ്യാൻ
- ഏജന്റുകൾക്ക് സ്വയം വിളിക്കാനുള്ള സൗകര്യം നൽകാൻ

ഈ ഡെമോയിൽ:
- **സ്റ്റോക്ക്ഹോം, സിയാറ്റിൽ, ടോക്കിയോ, ലണ്ടൻ, ആംസ്റ്റർഡാം** → റൂമുകൾ ലഭ്യമാണ് ✅
- **മറ്റു എല്ലാ നഗരങ്ങളും** → റൂമുകൾ ഇല്ല ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## ഘട്ടം 3: റൂട്ടിംഗ് നടത്താൻ കണ്ടീഷൻ ഫങ്ഷനുകൾ നിർവചിക്കുക

ഈ ഫങ്ഷനുകൾ ഏജന്റിന്റെ പ്രതികരണം പരിശോധിച്ച് പ്രവാഹത്തിൽ ഏത് പാത സ്വീകരിക്കണമെന്ന് നിർണ്ണയിക്കുന്നു.

**പ്രധാന മാതൃക:**
1. സന്ദേശം `AgentExecutorResponse` ആണോ എന്ന് പരിശോധിക്കുക  
2. ഘടനാപരമായ ഔട്ട്പുട്ട് (Pydantic മോഡൽ) പാഴ്സ് ചെയ്യുക  
3. റൂട്ടിംഗ് നിയന്ത്രിക്കാൻ `True` അല്ലെങ്കിൽ `False` മടക്കുക  

പ്രവാഹം **എഡ്ജുകളിൽ** ഈ നിബന്ധനകൾ വിലയിരുത്തി അടുത്ത എക്സിക്യൂട്ടർ ഏത് ആണെന്ന് തീരുമാനിക്കും.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## ഘട്ടം 4: കസ്റ്റം ഡിസ്പ്ലേ എക്സിക്യൂട്ടർ സൃഷ്ടിക്കുക

എക്സിക്യൂട്ടറുകൾ മാറ്റങ്ങൾ അല്ലെങ്കിൽ സൈഡ് ഇഫക്റ്റുകൾ നടത്തുന്ന വർക്ക്‌ഫ്ലോ ഘടകങ്ങളാണ്. അന്തിമ ഫലങ്ങൾ പ്രദർശിപ്പിക്കുന്ന ഒരു കസ്റ്റം എക്സിക്യൂട്ടർ സൃഷ്ടിക്കാൻ `@executor` ഡെക്കറേറ്റർ ഉപയോഗിക്കുന്നു.

**പ്രധാന ആശയങ്ങൾ:**
- `@executor(id="...")` - ഒരു ഫംഗ്ഷനെ വർക്ക്‌ഫ്ലോ എക്സിക്യൂട്ടറായി രജിസ്റ്റർ ചെയ്യുന്നു
- `WorkflowContext[Never, str]` - ഇൻപുട്ട്/ഔട്ട്പുട്ടിനുള്ള ടൈപ്പ് ഹിന്റുകൾ
- `ctx.yield_output(...)` - അന്തിമ വർക്ക്‌ഫ്ലോ ഫലം നൽകുന്നു


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## ഘട്ടം 5: പരിസ്ഥിതി ചോദ്യങ്ങൾ ലോഡ് ചെയ്യുക

LLM ക്ലയന്റ് ക്രമീകരിക്കുക. ഈ ഉദാഹരണം പ്രവർത്തിക്കുന്നത്:
- **GitHub മോഡലുകൾ** (GitHub ടോക്കൺ ഉപയോഗിച്ച് സൗജന്യ നിര)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## ഘട്ടം 6: ഘടനാപരമായ ഔട്ട്പുട്ടുകളോടെ AI ഏജന്റുമാരെ സൃഷ്ടിക്കുക

നാം **മൂന്ന് പ്രത്യേക ഏജന്റുമാരെ** സൃഷ്ടിക്കുന്നു, ഓരോന്നും `AgentExecutor` ഉപയോഗിച്ച് പാക്ക് ചെയ്യുന്നു:

1. **availability_agent** - ടൂൾ ഉപയോഗിച്ച് ഹോട്ടൽ ലഭ്യത പരിശോധിക്കുന്നു  
2. **alternative_agent** - (മുറികൾ ലഭ്യമല്ലെങ്കിൽ) പകരം നഗരങ്ങൾ നിർദ്ദേശിക്കുന്നു  
3. **booking_agent** - (മുറികൾ ലഭ്യമായാൽ) ബുക്കിംഗ് പ്രോത്സാഹിപ്പിക്കുന്നു  

**പ്രധാന സവിശേഷതകൾ:**
- `tools=[hotel_booking]` - ഏജന്റിന് ടൂൾ നൽകുന്നു  
- `response_format=PydanticModel` - ഘടനാപരമായ JSON ഔട്ട്പുട്ട് നിർബന്ധിതമാക്കുന്നു  
- `AgentExecutor(..., id="...")` - പ്രവൃത്തി പ്രവാഹത്തിൽ ഉപയോഗിക്കാൻ ഏജന്റിനെ പാക്ക് ചെയ്യുന്നു  


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## ഘട്ടം 7: വ്യവസ്ഥാപരമായ എഡ്ജുകളോടെ പ്രവൃത്തി പ്രവാഹം നിർമ്മിക്കുക

ഇപ്പോൾ നാം `WorkflowBuilder` ഉപയോഗിച്ച് വ്യവസ്ഥാപരമായ റൂട്ടിംഗ് ഉള്ള ഗ്രാഫ് നിർമ്മിക്കുന്നു:

**പ്രവൃത്തി പ്രവാഹത്തിന്റെ ഘടന:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**പ്രധാന രീതികൾ:**
- `.set_start_executor(...)` - പ്രവേശന ബിന്ദു സജ്ജമാക്കുന്നു
- `.add_edge(from, to, condition=...)` - വ്യവസ്ഥാപരമായ എഡ്ജ് ചേർക്കുന്നു
- `.build()` - പ്രവൃത്തി പ്രവാഹം അന്തിമമാക്കുന്നു


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## ഘട്ടം 8: ടെസ്റ്റ് കേസ് 1 നടത്തുക - ലഭ്യത ഇല്ലാത്ത നഗരം (പാരിസ്)

**ലഭ്യത ഇല്ലാത്ത** പാത പരിശോധിക്കാൻ, പാരിസിലെ ഹോട്ടലുകൾ (ഞങ്ങളുടെ സിമുലേഷനിൽ മുറികൾ ഇല്ലാത്തത്) അഭ്യർത്ഥിച്ച് പരീക്ഷിക്കാം.


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## ഘട്ടം 9: ടെസ്റ്റ് കേസ് 2 പ്രവർത്തിപ്പിക്കുക - ലഭ്യതയുള്ള നഗരം (സ്റ്റോക്ക്‌ഹോം)

ഇപ്പോൾ സ്റ്റോക്ക്‌ഹോം (ഞങ്ങളുടെ സിമുലേഷനിൽ മുറികൾ ഉള്ളത്) എന്ന നഗരത്തിൽ ഹോട്ടലുകൾ ആവശ്യപ്പെട്ട് **ലഭ്യത** പാത പരിശോധിക്കാം.


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## പ്രധാനപ്പെട്ട കാര്യങ്ങൾയും അടുത്ത ചുവടുകളും

### ✅ നിങ്ങൾ പഠിച്ചതെന്താണ്:

1. **WorkflowBuilder പാറ്റേൺ**
   - പ്രവേശന ബിന്ദു നിർവചിക്കാൻ `.set_start_executor()` ഉപയോഗിക്കുക
   - വ്യത്യസ്ത മാർഗ്ഗങ്ങൾക്കായി `.add_edge(from, to, condition=...)` ഉപയോഗിക്കുക
   - workflow പൂർത്തിയാക്കാൻ `.build()` വിളിക്കുക

2. **കണ്ടീഷണൽ റൂട്ടിംഗ്**
   - Condition ഫങ്ഷനുകൾ `AgentExecutorResponse` പരിശോധിക്കുന്നു
   - ഘടനാപരമായ ഔട്ട്പുട്ടുകൾ വിശകലനം ചെയ്ത് റൂട്ടിംഗ് തീരുമാനങ്ങൾ എടുക്കുക
   - ഒരു മാർഗ്ഗം സജീവമാക്കാൻ `True` മടക്കുക, ഒഴിവാക്കാൻ `False` മടക്കുക

3. **ടൂൾ ഇന്റഗ്രേഷൻ**
   - Python ഫങ്ഷനുകൾ AI ടൂളുകളാക്കി മാറ്റാൻ `@ai_function` ഉപയോഗിക്കുക
   - ഏജന്റുകൾക്ക് ആവശ്യമായപ്പോൾ ടൂളുകൾ സ്വയം വിളിക്കാം
   - ടൂളുകൾ ഏജന്റുകൾക്ക് വിശകലനം ചെയ്യാൻ കഴിയുന്ന JSON മടക്കുന്നു

4. **ഘടനാപരമായ ഔട്ട്പുട്ടുകൾ**
   - ടൈപ്പ്-സേഫ് ഡാറ്റ എക്സ്ട്രാക്ഷനായി Pydantic മോഡലുകൾ ഉപയോഗിക്കുക
   - ഏജന്റുകൾ സൃഷ്ടിക്കുമ്പോൾ `response_format=MyModel` സജ്ജമാക്കുക
   - `Model.model_validate_json()` ഉപയോഗിച്ച് പ്രതികരണങ്ങൾ വിശകലനം ചെയ്യുക

5. **കസ്റ്റം എക്സിക്യൂട്ടറുകൾ**
   - Workflow ഘടകങ്ങൾ സൃഷ്ടിക്കാൻ `@executor(id="...")` ഉപയോഗിക്കുക
   - എക്സിക്യൂട്ടറുകൾ ഡാറ്റ മാറ്റുകയോ സൈഡ് ഇഫക്റ്റുകൾ നടത്തുകയോ ചെയ്യാം
   - Workflow ഫലങ്ങൾ ഉത്പാദിപ്പിക്കാൻ `ctx.yield_output()` ഉപയോഗിക്കുക

### 🚀 യാഥാർത്ഥ്യപ്രധാനമായ പ്രയോഗങ്ങൾ:

- **ട്രാവൽ ബുക്കിംഗ്**: ലഭ്യത പരിശോധിക്കുക, ബദലുകൾ നിർദ്ദേശിക്കുക, ഓപ്ഷനുകൾ താരതമ്യം ചെയ്യുക
- **കസ്റ്റമർ സർവീസ്**: പ്രശ്നത്തിന്റെ തരം, വികാരഭാവം, മുൻഗണന എന്നിവയുടെ അടിസ്ഥാനത്തിൽ റൂട്ടുചെയ്യുക
- **ഇ-കൊമേഴ്സ്**: സ്റ്റോക്ക് പരിശോധിക്കുക, ബദലുകൾ നിർദ്ദേശിക്കുക, ഓർഡറുകൾ പ്രോസസ് ചെയ്യുക
- **കണ്ടന്റ് മോഡറേഷൻ**: വിഷമതാ സ്കോറുകൾ, ഉപയോക്തൃ ഫ്ലാഗുകൾ എന്നിവയുടെ അടിസ്ഥാനത്തിൽ റൂട്ടുചെയ്യുക
- **അപ്രൂവൽ വർക്ഫ്ലോകൾ**: തുക, ഉപയോക്തൃ റോൾ, റിസ്ക് ലെവൽ എന്നിവയുടെ അടിസ്ഥാനത്തിൽ റൂട്ടുചെയ്യുക
- **മൾട്ടി-സ്റ്റേജ് പ്രോസസ്സിംഗ്**: ഡാറ്റയുടെ ഗുണനിലവാരവും പൂർണ്ണതയും അടിസ്ഥാനമാക്കി റൂട്ടുചെയ്യുക

### 📚 അടുത്ത ചുവടുകൾ:

- കൂടുതൽ സങ്കീർണ്ണമായ കണ്ടീഷനുകൾ ചേർക്കുക (പല മാനദണ്ഡങ്ങൾ)
- Workflow സ്റ്റേറ്റ് മാനേജ്മെന്റുമായി ലൂപ്പുകൾ നടപ്പിലാക്കുക
- പുനരുപയോഗിക്കാവുന്ന ഘടകങ്ങൾക്കായി സബ്-വർക്ഫ്ലോകൾ ചേർക്കുക
- യഥാർത്ഥ APIകളുമായി (ഹോട്ടൽ ബുക്കിംഗ്, ഇൻവെന്ററി സിസ്റ്റങ്ങൾ) ഇന്റഗ്രേറ്റ് ചെയ്യുക
- പിശക് കൈകാര്യം ചെയ്യലും ഫാള്ബാക്ക് പാതകളും ചേർക്കുക
- ബിൽറ്റ്-ഇൻ വിസ്വലൈസേഷൻ ടൂളുകൾ ഉപയോഗിച്ച് വർക്ഫ്ലോകൾ ദൃശ്യമാക്കുക


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**അസത്യവാദം**:  
ഈ രേഖ AI വിവർത്തന സേവനമായ [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് വിവർത്തനം ചെയ്തതാണ്. കൃത്യതയ്ക്കായി ഞങ്ങൾ ശ്രമിക്കുന്നുവെങ്കിലും, ഓട്ടോമേറ്റഡ് വിവർത്തനങ്ങളിൽ പിശകുകൾ അല്ലെങ്കിൽ തെറ്റായ വിവരങ്ങൾ ഉണ്ടാകാൻ സാധ്യതയുണ്ട്. അതിന്റെ സ്വാഭാവിക ഭാഷയിലുള്ള മൗലിക രേഖയാണ് വിശ്വസനീയമായ ഉറവിടമായി കണക്കാക്കേണ്ടത്. നിർണായകമായ വിവരങ്ങൾക്ക്, പ്രൊഫഷണൽ മനുഷ്യ വിവർത്തനം ശുപാർശ ചെയ്യുന്നു. ഈ വിവർത്തനം ഉപയോഗിക്കുന്നതിൽ നിന്നുണ്ടാകുന്ന തെറ്റിദ്ധാരണകൾക്കോ തെറ്റായ വ്യാഖ്യാനങ്ങൾക്കോ ഞങ്ങൾ ഉത്തരവാദികളല്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
